# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Means

In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.spark.clustering.patchwork.PatchWork
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
// import clustering4ever.scala.clusterizables.RealClusterizable
// import clustering4ever.scala.vectorizables.RealVector

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.spark.clustering.patchwork.PatchWork
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2018-10-27 20:37:11--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.3’

     0K .......                                               100% 1010M=0s

2018-10-27 20:37:11 (1010 MB/s) - ‘/tmp/aggregation.csv.3’ saved [8063/8063]

:sh: Scheme missing.
--2018-10-27 20:37:11--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-10-27 20:37:11--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.4’

     0K .                                                     100

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val datasetSize = 1000000
val dim = 2
val path = "/tmp/aggregation.csv"
val useAggregationDS = true

// val rawData = if( useAggregationDS ) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer
//   else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))

// val parData = rawData.zipWithIndex.par.map{ case (v, id) => new RealClusterizable(id.toLong, new RealVector(v)) }

// val groundTruePath = "/tmp/labels"

val rdd = sc.parallelize(Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toSeq).cache
// val rdd = sc.parallelize(mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))).cache
val rdd2 = rdd.map(_.toArray).cache

rdd.count
rdd2.count

datasetSize: Int = 1000000
dim: Int = 2
path: String = /tmp/aggregation.csv
useAggregationDS: Boolean = true
rdd: org.apache.spark.rdd.RDD[scala.collection.mutable.ArrayBuffer[Double]] = ParallelCollectionRDD[0] at parallelize at <console>:89
rdd2: org.apache.spark.rdd.RDD[Array[Double]] = MapPartitionsRDD[1] at map at <console>:91
res4: Long = 788


788

## Parameters 

In [ ]:
// val epsilon = Array(0.1, 0.1)
val epsilon = Array(1.5, 1.5)
val minPts = 1
val minCellInCluster = 20
val ratio = 0.0

epsilon: Array[Double] = Array(1.5, 1.5)
minPts: Int = 1
minCellInCluster: Int = 20
ratio: Double = 0.0


## Run and measure the algorithm time

In [ ]:
val t1 = System.nanoTime
// for( i <- 0 until 10 ) new PatchWork(epsilon, minPts, ratio, minCellInCluster).run(rdd2)
val model = new PatchWork(epsilon, minPts, ratio, minCellInCluster).run(rdd2)
val t2 = System.nanoTime

(t2 - t1) / 1000000000D

t1: Long = 5783952865474
model: clustering4ever.spark.clustering.patchwork.PatchWorkModel = clustering4ever.spark.clustering.patchwork.PatchWorkModel@10a97085
t2: Long = 5784052820016
res19: Double = 0.099954542


0.099954542

## Different ways to apply model to datasets

In [ ]:
val clusterized1 = rdd2.map( v => (model.predict(v).getID, v) ).collect

clusterized1: Array[(Int, Array[Double])] = Array((1,Array(15.55, 28.65)), (1,Array(14.9, 27.55)), (1,Array(14.45, 28.35)), (1,Array(14.15, 28.8)), (1,Array(13.75, 28.05)), (1,Array(13.35, 28.45)), (1,Array(13.0, 29.15)), (1,Array(13.45, 27.5)), (1,Array(13.6, 26.5)), (1,Array(12.8, 27.35)), (1,Array(12.4, 27.85)), (1,Array(12.3, 28.4)), (1,Array(12.2, 28.65)), (1,Array(13.4, 25.1)), (1,Array(12.95, 25.95)), (1,Array(12.9, 26.5)), (1,Array(11.85, 27.0)), (1,Array(11.35, 28.0)), (1,Array(11.15, 28.7)), (1,Array(11.25, 27.4)), (1,Array(10.75, 27.7)), (1,Array(10.5, 28.35)), (1,Array(9.65, 28.45)), (1,Array(10.25, 27.25)), (1,Array(10.75, 26.55)), (1,Array(11.7, 26.35)), (1,Array(11.6, 25.9)), (1,Array(11.9, 25.05)), (1,Array(12.6, 24.05)), (1,Array(11.9, 24.5)), (1,Array(11.1, 25.2)), (1,...

## Plot clustering results

In [ ]:
val rawData = clusterized1.map(_._2.toArray).toArray
val labels = clusterized1.map(_._1).toArray
plot(rawData, labels, '*', Palette.COLORS)

rawData: Array[Array[Double]] = Array(Array(15.55, 28.65), Array(14.9, 27.55), Array(14.45, 28.35), Array(14.15, 28.8), Array(13.75, 28.05), Array(13.35, 28.45), Array(13.0, 29.15), Array(13.45, 27.5), Array(13.6, 26.5), Array(12.8, 27.35), Array(12.4, 27.85), Array(12.3, 28.4), Array(12.2, 28.65), Array(13.4, 25.1), Array(12.95, 25.95), Array(12.9, 26.5), Array(11.85, 27.0), Array(11.35, 28.0), Array(11.15, 28.7), Array(11.25, 27.4), Array(10.75, 27.7), Array(10.5, 28.35), Array(9.65, 28.45), Array(10.25, 27.25), Array(10.75, 26.55), Array(11.7, 26.35), Array(11.6, 25.9), Array(11.9, 25.05), Array(12.6, 24.05), Array(11.9, 24.5), Array(11.1, 25.2), Array(10.55, 25.15), Array(10.05, 25.95), Array(9.35, 26.6), Array(9.3, 27.25), Array(9.2, 27.8), Array(7.5, 28.25), Array(8.55, 27.45), Ar...

Window(javax.swing.JFrame[frame0,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 1,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,37,1000x963,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x963,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

## Inspect performance metrics

In [ ]:
val trueLabels = Source.fromFile(groundTruePath).getLines.map(_.toInt).toBuffer.toParArray

val (_, trueLabelsFrom0) = ExternalIndexes.prepareLabels(trueLabels)

val predicts = clusterized22.map(_.clusterID.get).toParArray

val nmi = ExternalIndexes.nmi(trueLabelsFrom0, predicts, SQRT)

nmi

<console>:83: error: not found: value groundTruePath
       val trueLabels = Source.fromFile(groundTruePath).getLines.map(_.toInt).toBuffer.toParArray
                                        ^
<console>:87: error: not found: value clusterized22
       val predicts = clusterized22.map(_.clusterID.get).toParArray
                      ^


In [ ]:
val v = mutable.ArrayBuffer.fill(30)(scala.util.Random.nextDouble)
val range = (0 until 10000)
val t1 = System.nanoTime
for( i <- range ) v.zipWithIndex.map{ case (v, idx) => v / idx.toDouble }
val t2 = System.nanoTime
for( i <- range ) mutable.ArrayBuffer((0 until v.size).map{ idx => v(idx) / idx.toDouble }:_*)
val t3 = System.nanoTime


(t3 - t2) / (t2 - t1).toDouble

v: scala.collection.mutable.ArrayBuffer[Double] = ArrayBuffer(0.993718068641032, 0.48368518252656645, 0.564895158520323, 0.8273605776024493, 0.6487522194690977, 0.8632550374991614, 0.8295695610039237, 0.26604752834765877, 0.9137423336666955, 0.973097170195876, 0.9740415849320366, 0.7707337627707871, 0.05447315635676453, 0.9681138454632979, 0.5804585123201909, 0.8474334519717482, 0.6037207049419462, 0.1970604901339723, 0.3713699849074692, 0.9305946302867912, 0.7358481672945889, 0.1881468082834984, 0.02383491357293477, 0.6650416442176781, 0.8307948711250103, 0.36709866188655094, 0.9920281764290136, 0.8406818869326529, 0.6549790917047208, 0.8504850394499323)
range: scala.collection.immutable.Range = Range(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,...

0.8194022149048333

In [ ]:
Array.concat(Array(1), Array(2))

res18: Array[Int] = Array(1, 2)


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total